In [0]:
import sys
import os
sys.path.append(os.path.abspath('../utils/'))

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
import json
from dwh_utils import *
from transformation_utils import *

In [0]:
df = spark.read.table("acme_ecommerce.silver.orders")

In [0]:
df= df.groupBy(F.year("order_date").alias("year"), "category", "sub-category", "customer_name").agg(F.sum("profit").alias("profit"), F.sum("price").alias("sales"), F.sum("discount").alias("discount"), F.sum("quantity").alias("qty"))

df = round_value(spark=spark, df=df, column="profit", round_to=2)
df = round_value(spark=spark, df=df, column="sales", round_to=2)
df = round_value(spark=spark, df=df, column="discount", round_to=2)

In [0]:
write_to_delta_with_cdc_by_name(
    spark=spark, df=df, catalog="acme_ecommerce", schema="gold", table="product_order_summary", merge_keys=["year", "category", "sub-category", "customer_name"], append_only=False
)

In [0]:
%sql
SELECT year, ROUND(SUM(profit), 2) as profit FROM acme_ecommerce.gold.product_order_summary GROUP BY year ORDER BY year;

In [0]:
%sql
SELECT year, category, ROUND(SUM(profit), 2) as profit FROM acme_ecommerce.gold.product_order_summary GROUP BY year, category ORDER BY year, category;

In [0]:
%sql
SELECT customer_name, ROUND(SUM(profit), 2) as profit FROM acme_ecommerce.gold.product_order_summary GROUP BY customer_name ORDER BY profit DESC;

In [0]:
%sql
SELECT year, customer_name, ROUND(SUM(profit), 2) as profit FROM acme_ecommerce.gold.product_order_summary GROUP BY year, customer_name ORDER BY year ASC, profit DESC;